# 开始介绍Agent的简单实现

## 引入一个互联网查询的工具 Tavily，也可以用GOOGL的搜索工具，好像目前没有国内的搜索工具集成

In [9]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(
    max_results = 1,
    verbose = True,
)

search.invoke("今天中国A股的表现如何，沪指和深指分别是多少？")

[{'url': 'https://cj.sina.com.cn/article/norm_detail?url=https%3A%2F%2Ffinance.sina.com.cn%2Fjjxw%2F2024-01-31%2Fdoc-inafmiaf5777755.shtml&finpagefr=w_110', 'content': 'A股风云突变，抄底！ 1月31日，A股指数走弱。截至收盘，沪指跌1.48%，深证成指跌1.95%，创业板指跌0.66%。计算机、军工、医药、电子、文旅板块普跌，全市场下跌个股近5000只。  300ETF、嘉实沪深300ETF和华夏沪深300ETF今日成交额分别为61.57亿元、41.69亿元、45.45亿元、14.37亿元。此外，富时中国A50指数期货也在午后突然拉升，一度涨超1%。  多只AI概念股午后闪崩 多只AI概念股午后闪崩。科大国创、海天瑞声跌超15%，昆仑万维跌超12%，天娱数科、二六三等跌停。  露落地，市场进入经济数据和业绩真空期，临近3月政策预期升温，新的预期开始形成，资金开始着手新的布局，市场风险偏好或有望改善。此外，2023年12月工业企业盈利降幅继续收窄，美债利率虽有反弹但对A股估值影响有限，随着市场对经济增长的担忧逐渐缓A股风云突变，抄底！ 中国证券报-中证网 2024-01-31 16:36. 去App听语音播报. 打开APP. 1月31日，A股指数走弱。截至收盘，沪指跌1.48%，深证成指跌1.95\xa0...'}]

[{'url': 'https://cj.sina.com.cn/article/norm_detail?url=https%3A%2F%2Ffinance.sina.com.cn%2Fjjxw%2F2024-01-31%2Fdoc-inafmiaf5777755.shtml&finpagefr=w_110',
  'content': 'A股风云突变，抄底！ 1月31日，A股指数走弱。截至收盘，沪指跌1.48%，深证成指跌1.95%，创业板指跌0.66%。计算机、军工、医药、电子、文旅板块普跌，全市场下跌个股近5000只。  300ETF、嘉实沪深300ETF和华夏沪深300ETF今日成交额分别为61.57亿元、41.69亿元、45.45亿元、14.37亿元。此外，富时中国A50指数期货也在午后突然拉升，一度涨超1%。  多只AI概念股午后闪崩 多只AI概念股午后闪崩。科大国创、海天瑞声跌超15%，昆仑万维跌超12%，天娱数科、二六三等跌停。  露落地，市场进入经济数据和业绩真空期，临近3月政策预期升温，新的预期开始形成，资金开始着手新的布局，市场风险偏好或有望改善。此外，2023年12月工业企业盈利降幅继续收窄，美债利率虽有反弹但对A股估值影响有限，随着市场对经济增长的担忧逐渐缓A股风云突变，抄底！ 中国证券报-中证网 2024-01-31 16:36. 去App听语音播报. 打开APP. 1月31日，A股指数走弱。截至收盘，沪指跌1.48%，深证成指跌1.95\xa0...'}]

## 构造一个检索器工具 Retriever

In [10]:
import os

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

loader = WebBaseLoader("https://docs.smith.langchain.com/overview")

docs = loader.load()

documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)

vector = Chroma.from_documents(
    documents, 
    OpenAIEmbeddings(
        openai_api_key = os.getenv("OPENAI_API_KEY"),
        base_url = os.getenv("OPENAI_BASE_URL")
    )
)
retriever = vector.as_retriever()

In [3]:
retriever.get_relevant_documents("how to upload a dataset")[0]

Document(page_content="dataset uploading.Once we have a dataset, how can we use it to test changes to a prompt or chain? The most basic approach is to run the chain over the data points and visualize the outputs. Despite technological advancements, there still is no substitute for looking at outputs by eye. Currently, running the chain over the data points needs to be done client-side. The LangSmith client makes it easy to pull down a dataset and then run a chain over them, logging the results to a new project associated with the dataset. From there, you can review them. We've made it easy to assign feedback to runs and mark them as correct or incorrect directly in the web app, displaying aggregate statistics for each test project.We also make it easier to evaluate these runs. To that end, we've added a set of evaluators to the open-source LangChain library. These evaluators can be specified when initiating a test run and will evaluate the results once the test run completes. If we’re 

### 构造一个Agent可以使用的tool

In [11]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    # 这里相当于一个工具的使用说明，agent在选择工具时会检查这个说明
    "Search for information about LangSmith.For any questions about LangSmith, you must use this tool"
)

tools = [search,retriever_tool]


## 构造一个简单的、没有memory的Agent

### 加载一个聊天模型包装器

In [12]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    temperature=0,
    openai_api_key = os.getenv("OPENAI_API_KEY"),
    base_url = os.getenv("OPENAI_BASE_URL")
)

## 可以到LangSmith的hub里面找别人已经写好的提示词

In [13]:
from langchain import hub

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")

print(prompt.messages[0].prompt.template)

You are a helpful assistant


## 构造一个Agent以及 Agent_executor

In [15]:
from langchain.agents import create_openai_functions_agent, AgentExecutor

agent = create_openai_functions_agent(llm, tools, prompt)

agent_executor = AgentExecutor(agent = agent, tools = tools)

agent_executor.invoke({"input":"what is the weather condition today in Tokyo？"})

[{'url': 'https://weatherspark.com/h/m/143809/2024/1/Historical-Weather-in-January-2024-in-Tokyo-Japan', 'content': 'January 2024 Weather History in Tokyo Japan  Cloud Cover in January 2024 in Tokyo Observed Weather in January 2024 in Tokyo  Humidity Comfort Levels in January 2024 in Tokyo Wind Speed in January 2024 in Tokyo  Tokyo Temperature History January 2024 Hourly Temperature in January 2024 in Tokyo Compare Tokyo to another city:January 2024 Weather History in Tokyo Japan The data for this report comes from the Tokyo International Airport. See all nearby weather stations Latest Report — 5:00 AM Tue, Jan 16, 2024 16 min ago UTC 20:00 Call Sign RJTT Temp. 36 °F very cold Precipitation No Report Wind 3.5 mph light air Wind Dir. 320 deg, NW Cloud Cover Mostly Clear 3,000 ft'}]

{'input': 'what is the weather condition today in Tokyo？',
 'output': 'The weather condition in Tokyo today is very cold with a temperature of 36 °F. There is no precipitation and the wind speed is 3.5 mph. The wind direction is 320 degrees, coming from the northwest. The cloud cover is mostly clear with clouds at 3,000 feet.'}

## 添加verbose，测试工具使用能力

In [16]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

agent_executor = AgentExecutor(
    agent = agent, 
    tools = tools,
    verbose =True,
)

agent_executor.invoke({"input":"what is the weather condition today in Tokyo？"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'weather condition today in Tokyo'}`


[{'url': 'https://weatherspark.com/h/m/143809/2024/1/Historical-Weather-in-January-2024-in-Tokyo-Japan', 'content': 'January 2024 Weather History in Tokyo Japan  Cloud Cover in January 2024 in Tokyo Observed Weather in January 2024 in Tokyo  Humidity Comfort Levels in January 2024 in Tokyo Wind Speed in January 2024 in Tokyo  Tokyo Temperature History January 2024 Hourly Temperature in January 2024 in Tokyo Compare Tokyo to another city:January 2024 Weather History in Tokyo Japan The data for this report comes from the Tokyo International Airport. See all nearby weather stations Latest Report — 5:00 AM Tue, Jan 16, 2024 16 min ago UTC 20:00 Call Sign RJTT Temp. 36 °F very cold Precipitation No Report Wind 3.5 mph light air Wind Dir. 320 deg, NW Cloud Cover Mostly Clear 3,000 ft'}]The weather condition in Tokyo today is very cold with a temperature of 36

{'input': 'what is the weather condition today in Tokyo？',
 'output': 'The weather condition in Tokyo today is very cold with a temperature of 36 °F. There is no precipitation and the wind speed is 3.5 mph. The wind direction is 320 degrees, coming from the northwest. The cloud cover is mostly clear with clouds at 3,000 feet.'}

In [17]:
agent_executor.invoke({"input":"你能告诉我今天沪指表现如何么?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': '今天沪指表现'}`


[{'url': 'https://finance.sina.com.cn/jjxw/2024-01-31/doc-inafmaua1646171.shtml', 'content': '1月31日，市场全天冲高回落，沪指跌1.48%再度失守2800点，两市超百股跌停。截至收盘，沪指跌1.48%，深成指跌1.95%，创业板指跌0.66%。  三大指数均收出月线六连阴，沪指本月跌超6%，深成指本月跌超13%，创业板指本月累计跌超16%，科创50指数本月跌近20%。 板块方面，仅出版板块逆势上涨；复合集流体、北斗导航、算力租赁、机器视觉等板块跌幅居前。  至于银行板块，无疑也是中字头+高股息的香饽饽，而今天红利指数的走强，其实也在提示这层关系。 沪指月线6连阴后，止跌将近还是更远了？ 数据显示，2023年8月以来，A股主要指数月线已报收六连阴。  昨天推送中我们提及的“沪指月线6连阴”已成事实。当“公认”的底线被反复挑战，市场距离真正止跌还有多远？ “中字头”之外，业绩预喜成新抱团方向2 hours ago — 2 hours ago1月31日，市场全天冲高回落，沪指跌1.48%再度失守2800点，两市超百股跌停。截至收盘，沪指跌1.48%，深成指跌1.95%，创业板指跌0.66%。'}]根据最新的数据，今天沪指下跌了1.48%，再次失守2800点，两市超过百股跌停。深成指下跌了1.95%，创业板指下跌了0.66%。沪指本月跌幅超过6%，深成指本月跌幅超过13%，创业板指本月累计跌幅超过16%。具体的行情详情可以参考[这篇文章](https://finance.sina.com.cn/jjxw/2024-01-31/doc-inafmaua1646171.shtml)。

> Finished chain.


{'input': '你能告诉我今天沪指表现如何么?',
 'output': '根据最新的数据，今天沪指下跌了1.48%，再次失守2800点，两市超过百股跌停。深成指下跌了1.95%，创业板指下跌了0.66%。沪指本月跌幅超过6%，深成指本月跌幅超过13%，创业板指本月累计跌幅超过16%。具体的行情详情可以参考[这篇文章](https://finance.sina.com.cn/jjxw/2024-01-31/doc-inafmaua1646171.shtml)。'}

In [23]:
agent_executor.invoke({"input":"could you tell me the latest index of NASDAQ?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'latest index of NASDAQ'}`


[{'url': 'https://www.bloomberg.com/quote/NDX:IND', 'content': '21,1998 the Nasdaq 100 was a cap-weighted index.  The NASDAQ-100 Index is a modified capitalization-weighted index of the 100 largest and most active non-financial  non-financial domestic and international issues listed on the NASDAQ. No security can have more than a 24% weighting.  markets news information as it happens.Index performance for NASDAQ 100 Stock Index (NDX) including value, chart, profile & other market data.'}]The latest index of NASDAQ is the NASDAQ 100 Stock Index (NDX). As of the latest information, the index is at 21,1998. You can find more information about the index on the [Bloomberg website](https://www.bloomberg.com/quote/NDX:IND).

> Finished chain.


{'input': 'could you tell me the latest index of NASDAQ?',
 'output': 'The latest index of NASDAQ is the NASDAQ 100 Stock Index (NDX). As of the latest information, the index is at 21,1998. You can find more information about the index on the [Bloomberg website](https://www.bloomberg.com/quote/NDX:IND).'}

In [24]:
agent_executor.invoke({"input":"告诉我langsimth的基本概念?"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'LangSmith基本概念'}`


[Document(page_content='LangSmith Overview and User Guide | 🦜️🛠️ LangSmith', metadata={'description': 'Building reliable LLM applications can be challenging. LangChain simplifies the initial setup, but there is still work needed to bring the performance of prompts, chains and agents up the level where they are reliable enough to be used in production.', 'language': 'en', 'source': 'https://docs.smith.langchain.com/overview', 'title': 'LangSmith Overview and User Guide | 🦜️🛠️ LangSmith'}), Document(page_content='Skip to main content🦜️🛠️ LangSmith DocsPython DocsJS/TS DocsSearchGo to AppLangSmithOverviewTracingTesting & EvaluationOrganizationsHubLangSmith CookbookOverviewOn this pageLangSmith Overview and User GuideBuilding reliable LLM applications can be challenging. LangChain simplifies the initial setup, but there is still work needed to bring the performance of prompts, chains a

{'input': '告诉我langsimth的基本概念?',
 'output': 'LangSmith是一种用于构建可靠的LLM（Language Model Microservice）应用程序的工具。LLM应用程序的性能需要达到足够可靠的水平才能投入生产使用，而LangSmith旨在简化这一过程。它通过优化提示（prompts）、链（chains）和代理（agents）的性能，提供了一种有效的方式来使用LangSmith并最大化其效益。\n\nLangSmith具有以下主要功能和特点：\n- 跟踪（Tracing）：LangSmith默认启用跟踪功能，可以在后台跟踪应用程序的运行情况。可以记录所有的跟踪信息，可视化延迟和令牌使用统计数据，并在需要时进行故障排除。\n- 测试和评估（Testing & Evaluation）：LangSmith可以帮助扩展评估集的表面积，通过快速编辑示例并将其添加到数据集中，以改进模型的质量或降低成本。\n- 监控（Monitoring）：LangSmith可以用于监控应用程序的运行情况，记录所有的跟踪信息，并可视化延迟和令牌使用统计数据。还可以与运行关联反馈信息，以跟踪性能并识别问题。\n- 协作调试（Collaborative Debugging）：LangSmith提供了“分享”按钮，可以将链和LLM运行共享给其他人进行调试。\n- 收集示例（Collecting Examples）：LangSmith鼓励收集应用程序中出现的失败示例，以便测试未来的链版本。这样可以更好地了解链的潜在问题，并进行改进。\n\n更多关于LangSmith的详细信息和使用指南，请参考[LangSmith Overview and User Guide](https://docs.smith.langchain.com/overview)。'}